# Used for predicting ALSFRS_slope
see https://www.synapse.org/#!Synapse:syn2873386/wiki/ .
We assumed data is vectorized + clustered + 6 features were selected

In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from vectorizing_funcs import *
from modeling_funcs import *

## Revectorize the selected data
We now reload the metadata and the 6 attributes selected per cluster

In [2]:
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )
train_data_means = pickle.load( open('../train_data_means.pickle', 'rb') )
train_data_std = pickle.load( open('../train_data_std.pickle', 'rb') )
best_features_per_cluster = pickle.load( open('../best_features_per_cluster.pickle', 'rb') )


df = pd.read_csv('../train_data_selected.csv', sep='|', index_col=False, dtype="unicode")
vectorized, _ = vectorize(df, all_feature_metadata)
normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
print normalized.shape
normalized.head()

(1777, 62)


,family_ALS_hist_last,weight_pct_diff,weight_mean_slope,weight_mean,weight_median,weight_std,F,M,Age_last,respiratory_rate_pct_diff,...,bp_diastolic_mean,bp_diastolic_median,bp_diastolic_std,Creatinine_pct_diff,Creatinine_last,Creatinine_mean_slope,Creatinine_mean,Creatinine_median,Creatinine_std,fvc_percent_pct_diff
SubjectID,,,,,,,,,,,,,,,,,,,,,
649,0,0,0,0,0,0,1.322796,-1.322796,-0.558473,0,...,0,0,0,0,0,0,0,0,0,-0.037980
2492,0,0,0,0,0,0,-0.755549,0.755549,0.764854,0,...,0,0,0,0,0,0,0,0,0,-0.361454
3085,0,0,0,0,0,0,-0.755549,0.755549,-1.617135,0,...,0,0,0,0,0,0,0,0,0,0.185905
3551,0,0,0,0,0,0,-0.755549,0.755549,0.059079,0,...,0,0,0,0,0,0,0,0,0,0.230714
4390,0,0,0,0,0,0,-0.755549,0.755549,-1.528913,0,...,0,0,0,0,0,0,0,0,0,0.253000


In [3]:
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col="SubjectID")
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col="SubjectID")

clusters.index = clusters.index.astype(str)
slope.index = slope.index.astype(str)
normalized.index = normalized.index.astype(str)

X = normalized.join(clusters)
Y = slope.join(clusters)

print Y.shape, X.shape, clusters.shape
display(X.head(3))
display(Y.head(3))


(1777, 2) (1777, 63) (1777, 1)


,family_ALS_hist_last,weight_pct_diff,weight_mean_slope,weight_mean,weight_median,weight_std,F,M,Age_last,respiratory_rate_pct_diff,...,bp_diastolic_median,bp_diastolic_std,Creatinine_pct_diff,Creatinine_last,Creatinine_mean_slope,Creatinine_mean,Creatinine_median,Creatinine_std,fvc_percent_pct_diff,cluster
SubjectID,,,,,,,,,,,,,,,,,,,,,
649,0,0,0,0,0,0,1.322796,-1.322796,-0.558473,0,...,0,0,0,0,0,0,0,0,-0.037980,0
2492,0,0,0,0,0,0,-0.755549,0.755549,0.764854,0,...,0,0,0,0,0,0,0,0,-0.361454,0
3085,0,0,0,0,0,0,-0.755549,0.755549,-1.617135,0,...,0,0,0,0,0,0,0,0,0.185905,0


,ALSFRS_slope,cluster
SubjectID,,
533,-0.965608,2
649,-0.921717,0
1234,-0.914787,1


## Train a prediction model per cluster

In [4]:
model_per_cluster = get_model_per_cluster(X, Y)
    

cluster: 0 size: (620L,)
Mean square error (0 is perfect): 0.42
Explained variance score (1 is perfect): 0.00

cluster: 1 size: (530L,)
Mean square error (0 is perfect): 0.33
Explained variance score (1 is perfect): 0.03

cluster: 2 size: (627L,)
Mean square error (0 is perfect): 0.40
Explained variance score (1 is perfect): 0.01



In [5]:
with open("../model_per_cluster.pickle", "wb") as output_file:
    pickle.dump(model_per_cluster, output_file)


## Apply the model on both `train` and `test`

In [6]:

for t in ['train', 'test']:
    df = pd.read_csv('../' + t + '_data_selected.csv', sep='|', index_col=False)
    vectorized, _ = vectorize(df, all_feature_metadata)
    normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
    
    clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col=0)

    clusters.index = clusters.index.astype(str)
    normalized.index = normalized.index.astype(str)

    X = normalized.join(clusters)
    pred = X.apply(apply_model, args=[model_per_cluster], axis = 1)
    pred.to_csv('../' + t + '_prediction.csv',sep='|')

pred.head()

,cluster,prediction
SubjectID,,
750059,0,-0.715995
750195,0,-0.715995
750406,0,-0.715995
755652,0,-0.715995
756895,0,-0.715995
